# STAC EuroSAT

This notebook demonstrates how to convert annotations provided by [EuroSAT](https://github.com/phelber/EuroSAT) dataset
into STAC-compatible definitions with extensions relevant for machine learning tasks.
Notably, the STAC [Label](https://github.com/stac-extensions/label)and [Scientific](https://github.com/stac-extensions/scientific)
extensions are used to reference the labeled annotations from train, validation and test splits, and provide citation reference to
the original work respectively.

To facilitate parsing of EuroSAT metadata itself,
the [torchgeo.datasets.EuroSAT](https://torchgeo.readthedocs.io/en/stable/api/datasets.html#torchgeo.datasets.EuroSAT)
class will be used to handle the metadata extraction process, parsing the labeled data hierarchy, and generate the splits
definition and sample generation from them.

## First Step

Below are equivalent `torchgeo.datasets.EuroSAT100` and `torchgeo.datasets.EuroSAT` classes that define a subset and the complete dataset
respectively. While developing or editing the STAC generation pipeline, it is recommended to work with the 100 subset variation to speed up
the process and directly observe an overview of the expected result.

In [1]:
from torchgeo.datasets import EuroSAT, EuroSAT100  # noqa

# pick one:
DatasetEuroSAT = EuroSAT100  # subset of (6 train, 2 val, 2 test) image samples per class (10)
# DatasetEuro = EuroSAT100  # full dataset

## General configurations

In [2]:
import os

DATA_ROOT_DIR = os.path.abspath("../data")
CATALOG_ROOT_DIR = DATA_ROOT_DIR
EUROSAT_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "EuroSAT")
EUROSAT_TYPE_DIR = "subset" if issubclass(EuroSAT100, DatasetEuroSAT) else "full"  # need distinct dirs, otherwise download dir name clash
EUROSAT_DATA_DIR = os.path.join(EUROSAT_ROOT_DIR, "data", EUROSAT_TYPE_DIR)
EUROSAT_STAC_DIR = os.path.join(EUROSAT_ROOT_DIR, "stac")
EUROSAT_STAC_URL = "https://example.com/"  # base URL where samples would be accessible from (links in STAC)

os.makedirs(EUROSAT_DATA_DIR, exist_ok=True)
os.makedirs(EUROSAT_STAC_DIR, exist_ok=True)

## STAC Definitions

Following types are **NOT** used for "strong" type checking.
They are provided as reference of the expected STAC properties and for quick validation of the structure by IDEs.

### Base Definitions

In [3]:
import datetime
import os
from typing import Any, Dict, List, Literal, Optional, TypedDict, Union
from typing_extensions import NotRequired, Required


STAC_VERSION = "1.0.0"

STAC_CATALOG_EXTENSIONS = []
STAC_CATALOG_SCHEMAS = [
    f"https://schemas.stacspec.org/v{STAC_VERSION}/catalog-spec/json-schema/catalog.json",
] + STAC_CATALOG_EXTENSIONS

STAC_COLLECTION_EXTENSIONS = [
    "https://stac-extensions.github.io/eo/v1.1.0/schema.json",
    "https://stac-extensions.github.io/ml-aoi/v0.1.0/schema.json",
    "https://stac-extensions.github.io/scientific/v1.0.0/schema.json",
    "https://stac-extensions.github.io/version/v1.0.0/schema.json",
]
STAC_COLLECTION_SCHEMAS = [
    f"https://schemas.stacspec.org/v{STAC_VERSION}/collection-spec/json-schema/collection.json",
] + STAC_COLLECTION_EXTENSIONS

STAC_ITEM_EXTENSIONS = [
    "https://stac-extensions.github.io/eo/v1.1.0/schema.json",
    "https://stac-extensions.github.io/file/v1.0.0/schema.json",
    "https://stac-extensions.github.io/label/v1.0.1/schema.json",
    "https://stac-extensions.github.io/ml-aoi/v0.1.0/schema.json",
    "https://stac-extensions.github.io/version/v1.0.0/schema.json",
]
STAC_ITEM_SCHEMAS = [
    f"https://schemas.stacspec.org/v{STAC_VERSION}/item-spec/json-schema/item.json",
] + STAC_ITEM_EXTENSIONS

# technically, tuples would be better for bbox/point, but not the types used in JSON
Number = Union[int, float]
BoundingBox = List[Number]  # 4 value
Point = List[Number]  # 2 values
DateTimeInterval = List[Union[str, None]]
Split = Literal["train", "validate", "test"]
GeoJSONProperties = TypedDict(
    "GeoJSONProperties",
    {
        "class": str,
    },
    total=False,
)
GeoJSONGeometry = TypedDict(
    "GeoJSONGeometry",
    {
        "type": Literal["Polygon", "MultiPolygon"],  # others exist, but not really applicable for this case
        "properties": NotRequired[GeoJSONProperties],
        "coordinates": List[Point],  # at least 4 normally for square/polygon bbox, but more valid if multi-polygon
    }
)
SpatialExtent = TypedDict(
    "SpatialExtent",
    {
        "bbox": Required[List[BoundingBox]],
    }
)
TemporalExtent = TypedDict(
    "TemporalExtent",
    {
        "interval": Required[List[DateTimeInterval]],
    }
)
Extent = TypedDict(
    "Extent",
    {
        "spatial": Required[SpatialExtent],
        "temporal": Required[TemporalExtent],
    }
)
Provider = TypedDict(
    "Provider",
    {
        "name": str,
        "roles": List[str],
        "url": str,
    }
)
Link = TypedDict(
    "Link",
    {
        "rel": str,
        "href": str,
        "type": str,  # media-type
        "title": NotRequired[str],
    },
    total=False,
)
STACMetadata = TypedDict(
    "STACMetadata",
    {
        "stac_version": Required[str],
        "type": Required[Literal["Catalog", "Collection", "Feature"]],  # NB: Feature == STAC Item
        "id": Required[str],
        "title": NotRequired[str],
        "description": NotRequired[str],
        "links": Required[List[Link]],
    }
)

### STAC Extensions

In [4]:
STACExtensionVersion = TypedDict(
    "STACExtensionVersion",
    {
        "version": Required[str],
        "deprecated": NotRequired[bool],
        "experimental": NotRequired[bool],
    }
)
STACLabelRef = TypedDict(
    "STACLabelRef",
    {
        "title": str,
        "href": str,    # URL to GeoJSON FeatureCollection or GeoTiff/COG
        "type": str,    # media-type
    }
)
STACLabelAssets = TypedDict(
    "STACLabelAssets",
    {
        "labels": STACLabelRef,
        "raster": STACLabelRef,
    }
)
STACLabelClass = TypedDict(
    "STACLabelClass",
    {
        # name that can define a "category" of classe names
        # those categories should be specified in Features under the keys defined by 'label:properties'
        "name": Required[Union[str, None]],
        # all the applicable classes that should be part of the "category"
        "classes": Required[Union[List[str], List[int]]],
    }
)
STACLabelCount = TypedDict(
    "STACLabelCount",
    {
        "name": str,  # class
        "count": int,
    }
)
STACLabelOverview = TypedDict(
    "STACLabelOverview",
    {
        "property_key": str,
        "counts": List[STACLabelCount],
    }
)
STACLabelProperties = TypedDict(
    "STACLabelProperties",
    {
        # properties in the linked 'labels' asset with GeoJSON FeatureCollection
        # those properties should be provided for each Feature in the GeoJSON
        # the values of those properties should contain the keys from 'label:classes'
        "label:properties": Required[List[str]],
        "label:classes": Required[List[STACLabelClass]],
        "label:type": Required[Literal["raster", "vector"]],
        "label:description": Required[str],
        "label:methods": NotRequired[List[Literal["manual", "automatic"]]],
        "label:tasks": NotRequired[List[Literal["classification", "detection", "segmentation", "regression"]]],
        "label:overviews": NotRequired[List[Dict[str, Any]]],
    }
)
STACExtensionLabel = TypedDict(
    "STACExtensionLabel",
    {
        "assets": Required[STACLabelAssets],
        "properties": Required[STACLabelProperties],
    }
)
STACExtensionMLAOI = TypedDict(
    "STACExtensionMLAOI",
    {
        "ml-aoi:split": NotRequired[Split],
    }
)
STACEOAssets = TypedDict(
    "STACEOAssets",
    {
        "reflectance": NotRequired[Link],
        "temperature": NotRequired[Link],
        "saturation": NotRequired[Link],
        "cloud": NotRequired[Link],
        "cloud-shadow": NotRequired[Link],
    },
    total=False,
)
# Metadata about each band: https://github.com/stac-extensions/eo#common-band-names
STACEOBandCommonName = Literal[
    "coastal",
    "blue",
    "green",
    "red",
    "yellow",
    "pan",
    "rededge",
    "nir",
    "nir08",
    "nir09",
    "cirrus",
    "swir16",
    "swir22",
    "lwir",
    "lwir11",
    "lwir12",
]
# Definition of each field: https://github.com/stac-extensions/eo#band-object
STACEOBand = TypedDict(
    "STACEOBand",
    {
        "name": Required[str],
        "common_name": Required[STACEOBandCommonName],
        "description": NotRequired[str],
        "center_wavelength": NotRequired[Number],
        "full_width_half_max": NotRequired[Number],
        "solar_illumination": NotRequired[Number],
        "assets": NotRequired[STACEOAssets],
    }
)
STACExtensionEO = TypedDict(
    "STACExtensionEO",
    {
        "eo:bands": Required[List[STACEOBand]],
        "eo:cloud_cover": NotRequired[Number],
        "eo:snow_cover": NotRequired[Number],
    }
)
STACCitation = TypedDict(
    "STACCitation",
    {
        "doi": str,
        "citation": str,
    }
)
STACExtensionScientific = TypedDict(
    "STACExtensionScientific",
    {
        # how to cite this collection
        "sci:doi": Required[str],  # ++ "Link" with 'cite-as' using the DOI reference (RFC-8574)
        "sci:citation": Required[str],
        # related work/citations that use this STAC data collection
        "sci:publications": NotRequired[List[STACCitation]],
    }
)

### STAC Catalog

In [5]:
STACCatalog = STACMetadata  # only requires "links" to contain the STAC Collection as "child"

### STAC Collection and Extensions

In [6]:
# https://github.com/radiantearth/stac-spec/blob/master/item-spec/common-metadata.md#instrument
STACCoreInstrumentItem = TypedDict(
    "STACCoreInstrumentItem",
    {
        "platform": NotRequired[str],  # satellite
        "instruments": NotRequired[List[str]],
        "constellation": NotRequired[str],
        "mission": NotRequired[str],
        "gsd": NotRequired[Number],
    }
)
STACCoreItemProperties = TypedDict(
    "STACCoreItemProperties",
    {
        "datetime": Required[str],
        "license": Required[str],
    },
    total=False,
)
STACCoreFeatureItem = TypedDict(
    "STACCoreFeatureItem",
    {
        "type": Literal["Feature"],
        "bbox": Required[BoundingBox],
        "geometry": Required[GeoJSONGeometry],
        "assets": Required[List[Dict[str, Any]]],
        "properties": Required[STACCoreItemProperties],
        "collection": str,
    }
)
STACExtendedItem = Union[
    STACMetadata,
    STACCoreInstrumentItem,
    STACCoreFeatureItem,
    STACExtensionEO,
    STACExtensionLabel,
    STACExtensionMLAOI,
]

In [7]:
STACMetadataCollection = TypedDict(
    "STACMetadataCollection",
    {
        "stac_extensions": Required[List[str]],
        "version": Required[str],
        "keywords": NotRequired[List[str]],
        "license": Required[str],  # anything, but commonly "CC-BY-SA-4.0"
    }
)
STACCoreCollection = TypedDict(
    "STACCoreCollection",
    {
        "extent": Required[Extent],
        "providers": NotRequired[List[Provider]],
    }
)
STACExtendedCollection = Union[
    STACMetadata,
    STACMetadataCollection,
    STACCoreCollection,
    STACExtensionMLAOI,
    STACExtensionScientific,
    STACExtensionVersion,
]

### STAC Item and Assets

### STAC Metadata Definition

To make the calling functions more succinct, start by defining constant metadata references used by STAC Collections.

In [8]:
EUROSAT_STAC_COLLECTION_BASE: STACExtendedCollection = {
    "stac_version": STAC_VERSION,
    "stac_extensions": STAC_COLLECTION_EXTENSIONS,
    "type": "Collection",
    "version": "1",
    "license": "MIT",  # https://github.com/phelber/EuroSAT#license
    "experimental": True,
    "sci:doi": "arXiv:1709.00029",  # https://github.com/phelber/EuroSAT#references
    "sci:citation": (
        "Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. "
        "Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. "
        "IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019."
    ),
    "sci:publications": [
        {
            "doi": "10.1109/IGARSS.2018.8519248",
            "citation": (
                "Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. "
                "Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 "
                "IEEE International Geoscience and Remote Sensing Symposium, 2018."
            ),
        }
    ],
    "links": [
        {
            "rel": "cite-as",
            "href": "https://arxiv.org/abs/1709.00029",
            "title": "EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification",
        }
    ]
    # other items to fill by script
}

## Populate STAC Catalog, Collections, Items and Assets from EuroSAT Dataset

**Note** <br>
Because we need the metadata to populate STAC Collections, Items and Assets, the `download=True` parameter is used.
However, we don't need the actual data (imagery pixel values), but instead the metadata that each GeoTiff contains.
Therefore, we override the `__getitem__` method to retrieve only metadata, by bypassing image loading/conversion, and make parsing faster.
Nevertheless, it can take some time to download and extract the ZIP contents on the first run.

In [14]:
import json
from copy import deepcopy
from datetime import datetime
from functools import cache

import requests
import numpy as np
import rasterio
from rasterio import MemoryFile
from rasterio.crs import CRS
from rasterio.warp import transform_bounds, transform_geom
from shapely.geometry import Polygon, box
from shapely.ops import unary_union
from PIL import Image
from PIL.Image import Resampling  # noqa  # IDE doesn't see it although correct location


RasterIOImage = MemoryFile
SampleMetadata = TypedDict(
    "SampleMetadata",
    {
        "image": str,
        "index": int,
        "label": str,
        "class": str,
    }
)

# required by STAC as standard for all Geo references
EPSG_4326 =  CRS.from_epsg(4326)  # type: ignore  # IDE type stub error


class DataLoaderEuroSAT(DatasetEuroSAT):
    @cache
    def __getitem__(
        self,
        sample_index: int,
    ) -> Optional[SampleMetadata]:  # type: ignore  # mismatch 'Dict[str, Tensor]' on purpose
        img_path, target_index = self.samples[sample_index]
        class_name = self.classes[target_index]
        return {"image": img_path, "index": sample_index, "label": str(target_index), "class": class_name}


def normalize_band(image_band: np.ndarray) -> np.ndarray:
    band_min, band_max = image_band.min(), image_band.max()  # type: ignore  # IDE type stub error
    return (image_band - band_min) / (band_max - band_min)


def brighten_band(image_band: np.ndarray, alpha: float = 0.13, beta: float = 0.0, gamma: float = 2.0) -> np.ndarray:
    return np.clip(np.power(alpha * image_band + beta, 1. / gamma), 0, 255)


def save_thumbnail(src_path: str, png_path: str) -> None:
    """
    Generate a visually adequate RBG preview from the relevant RGB bands.

    Since Sentinel-2 has 13 bands, they cannot all be loaded by traditional image utilities.
    """
    if not os.path.isfile(png_path):
        os.makedirs(os.path.dirname(png_path), exist_ok=True)

        indices = [
            DataLoaderEuroSAT.all_band_names.index(band) + 1  # 1-based indices for bands
            for band in DataLoaderEuroSAT.rgb_bands
        ]
        img = rasterio.open(src_path)
        rgb = Image.fromarray(
            (
                np.dstack([
                    brighten_band(normalize_band(img.read(idx))) for idx in indices
                ])
                * 255
            ).astype(np.uint8)
        )

        w_size = 64
        if rgb.size != (w_size, w_size):
            w_scale = w_size / float(img.size[0])
            h_size = int(float(img.size[1]) * float(w_scale))
            rgb = rgb.resize((w_size, h_size), Resampling.LANCZOS)

        rgb.save(png_path)


@cache
def get_stac_eo_details(loader: DataLoaderEuroSAT) -> Union[STACCoreInstrumentItem, STACExtensionEO]:
    # https://github.com/stac-extensions/eo#common-band-names
    # use the reference STAC Collection for Sentinel-2 to have all validated metadata of the bands and sensors
    data = requests.get("https://raw.githubusercontent.com/sat-utils/sat-stac-sentinel/master/stac_sentinel/sentinel-s2-l1c.json").json()
    stac_eo: Union[STACCoreInstrumentItem, STACExtensionEO] = data["properties"]
    for band in stac_eo["eo:bands"]:
        # fix mapping error between reference and EuroSAT dataset: B8A -> B08A
        if band["name"] == "B8A":
            band["name"] = "B08A"
        # add missing common name, as per recommendation in STAC EO common band names for Sentinel-2
        if "common_name" not in band:
            if band["name"] in ["B05", "B06", "B07"]:
                band["common_name"] = "rededge"
            if band["name"] == "B08A":
                band["common_name"] = "nir08"
            if band["name"] == "B09":
                band["common_name"] = "nir09"
    assert len(set(loader.all_band_names) - set(band["name"] for band in stac_eo["eo:bands"])) == 0, "Missing EO band definitions"
    assert all(band.get("common_name") for band in stac_eo["eo:bands"]), "Missing EO band common names"
    return stac_eo


def convert_sample_to_stac_item(
    sample: SampleMetadata,
    split: Split,
    loader: DataLoaderEuroSAT,
) -> STACExtendedItem:
    #img = gdal.Open(path)
    #geo = img.GetGeoTransform()
    img = rasterio.open(sample["image"])
    png_path = sample["image"].replace("/tif/", "/png/").replace(".tif", ".png")
    save_thumbnail(sample["image"], png_path)
    bbox_bounds = transform_bounds(
        img.crs,
        EPSG_4326,
        *img.bounds,
    )
    geom = transform_geom(
        img.crs,
        EPSG_4326,
        box(*bbox_bounds),
    )
    class_name = sample["class"]
    image_name = os.path.splitext(os.path.split(sample["image"])[-1])[0]
    label_geojson: GeoJSONGeometry = geom.__geo_interface__ if hasattr(geom, "__geo_interface__") else geom  # GeoJSON
    label_geojson["properties"] = {"class": class_name}
    label_path = sample["image"].replace("/tif/", "/label/").replace(".tif", ".geojson")
    os.makedirs(os.path.dirname(label_path), exist_ok=True)
    save_json(label_geojson, label_path)
    raster_link = {
        "title": f"Raster {image_name} with {class_name} class",
        "href": get_url(sample["image"]),
        "type": "image/tiff; application=geotiff",  # add "; profile=cloud-optimized" if applicable
    }
    source_link = raster_link.copy()
    source_link.update({
        "rel": "source",
        "label:assets": ["labels", "raster"],
    })
    thumbnail_base = {
        "title": f"Preview of {image_name}.",
        "href": get_url(png_path),
        "type": "image/png",
    }
    thumbnail_link = thumbnail_base.copy()
    thumbnail_link["rel"] = "thumbnail"
    stac_item: STACExtendedItem = {
        "stac_version": STAC_VERSION,
        "stac_extensions": STAC_ITEM_EXTENSIONS,
        "type": "Feature",
        "bbox": bbox_bounds,
        "geometry": label_geojson,  # GeoJSON
        "assets": {
            "labels": {
                "title": f"Labels for image {image_name} with {class_name} class",
                "href": get_url(label_path),
                "type": "application/geo+json"
            },
            "raster": raster_link,
            "thumbnail": thumbnail_base,
        },
        "properties": {
            "datetime": datetime.utcnow().isoformat(),
            "license": "MIT",  # https://github.com/phelber/EuroSAT#license
            "version": "1",
            "label:properties": [
                "class"
            ],
            "table:tasks": ["segmentation", "classification"],
            "label:type": "raster",
            "label:method": ["manual"],
            "label:description": "Land-cover area classification on Sentinel-2 image.",
            "label:classes": [
                {
                    "name": "class",
                    "classes": [sample["class"], sample["label"]],
                }
            ],
            "label:overviews": [
                # basic overview since each sample has its own STAC Item
                {
                    "property_key": "class",
                    "counts": [{"name": sample["class"], "count": 1}],
                }
            ],
            "ml-aoi:split": split,
            "ml-aoi:role": "label",
            "ml-aoi:reference-grid": True,
        },
        "links": [
            thumbnail_link,
            source_link,
            # STAC Catalog/Collection references added by caller
        ]
    }
    stac_eo = get_stac_eo_details(loader)
    stac_prop = stac_item["properties"]
    stac_prop.update(stac_eo)
    return stac_item


@cache
def get_url(url_path: str) -> str:
    url_path = url_path.replace(CATALOG_ROOT_DIR, "").lstrip("/")
    url_path = os.path.join(EUROSAT_STAC_URL, url_path)
    return url_path


def save_json(data: Dict[str, Any], json_file_path: str) -> None:
    with open(json_file_path, mode="w", encoding="utf-8") as fs:
        json.dump(data, fs, indent=2, ensure_ascii=False, sort_keys=False)


def load_json(json_file_path: str) -> Dict[str, Any]:
    with open(json_file_path, mode="r", encoding="utf-8") as fs:
        return json.load(fs)

In [22]:
from tqdm.notebook import tqdm


CATALOG_URL = get_url("catalog.json")
CATALOG_ROOT_LINK = {
  "rel": "root",
  "href": CATALOG_URL,
  "type": "application/json"
}


def generate_stac_collections():
    stac_catalog_collection_links: List[Link] = []
    stac_item_geometries: List[GeoJSONGeometry] = []
    for split_name in tqdm(DatasetEuroSAT.splits, desc="Split"):
        split: Split = "validate" if split_name == "val" else split_name
        stac_collection = EUROSAT_STAC_COLLECTION_BASE.copy()
        stac_collection["extent"] = {}  # temporary for ordering
        stac_collection.update({
            "ml-aoi:split": split,
        })
        stac_collection_path = os.path.join(EUROSAT_STAC_DIR, split, "collection.json")
        stac_collection_url = get_url(stac_collection_path)
        stac_collection_link: Link = {
            "rel": "collection",
            "href": stac_collection_url,
            "type": "application/json",
            "title":  f"EuroSAT STAC Collection with samples from '{split}' split.",
        }
        stac_collection_parent = stac_collection_link.copy()
        stac_collection_parent["rel"] = "parent"

        data_loader = DataLoaderEuroSAT(root=EUROSAT_DATA_DIR, split=split_name, download=True)
        os.makedirs(os.path.join(EUROSAT_STAC_DIR, split), exist_ok=True)
        for sample in tqdm(data_loader, desc=f"Sample ({split})"):
            label = sample["label"]
            if not label:  # ignore tiles by themselves without annotations
                continue
            index = sample["index"]
            iloc = os.path.join(EUROSAT_STAC_DIR, split, f"item-{index}.json")
            item = convert_sample_to_stac_item(sample, split, data_loader)
            stac_item_geometries.append(item["geometry"])

            # links for each respective item
            item.setdefault("links", [])
            item_link = {
                "rel": "item",
                "href": get_url(iloc),
                "type": "application/geo+json",
            }
            item_self = item_link.copy()
            item_self["rel"] = "self"
            item["links"].extend([
                CATALOG_ROOT_LINK,
                stac_collection_parent,
                stac_collection_link,
                item_self,
            ])

            stac_collection["links"].append(item_link)
            save_json(item, iloc)

        stac_collection["extent"] = {}

        # full extent considering all samples
        extent_geom = unary_union([Polygon(*poly["coordinates"]) for poly in stac_item_geometries])
        stac_collection["extent"]["spatial"] = {"bbox": [extent_geom.bounds]}
        # FIXME  search paper for range
        #   https://www.researchgate.net/publication/319463676_EuroSAT_A_Novel_Dataset_and_Deep_Learning_Benchmark_for_Land_Use_and_Land_Cover_Classification
        # otherwise, Sentinel-2 collection provides ["2015-06-27T10:25:31.456Z", null]
        # limit from 2015 up to paper publication date?
        stac_collection["extent"]["temporal"] = {"interval": ["2015-06-27T10:25:31.456Z", None]}  # FIXME

        # more link references
        stac_collection_self = stac_collection_link.copy()
        stac_collection_self["rel"] = "self"
        stac_catalog_collection_links.append(stac_collection_self)
        stac_collection["links"].extend([
            CATALOG_ROOT_LINK,
            stac_collection_link,
            {
                "rel": "parent",
                "href": get_url("catalog.json"),
                "type": "application/json",
                "title": "STAC Catalog",
            }
        ])
        stac_eo = get_stac_eo_details(data_loader)
        stac_collection.setdefault("properties", {})
        stac_collection["properties"].update(stac_eo)
        save_json(stac_collection, stac_collection_path)

    return stac_catalog_collection_links

In [23]:
stac_catalog_collections = generate_stac_collections()
catalog_collections_refs = deepcopy(stac_catalog_collections)
for link in catalog_collections_refs:
    link["rel"] = "child"


catalog: STACCatalog = {
    "id": "example",
    "type": "Catalog",
    "title": "Example STAC Catalog",
    "stac_version": STAC_VERSION,
    "description": "Example catalog with annotated label collections.",
    "links": [
         CATALOG_ROOT_LINK,
        {
            "rel": "self",
            "href": CATALOG_URL,
            "type": "application/json"
        }
    ] + catalog_collections_refs
}

Split:   0%|          | 0/3 [00:00<?, ?it/s]

Sample (train):   0%|          | 0/60 [00:00<?, ?it/s]

Sample (validate):   0%|          | 0/20 [00:00<?, ?it/s]

Sample (test):   0%|          | 0/20 [00:00<?, ?it/s]

## STAC Schema Validation

Verify that all items that were generated respect the various STAC schemas.

In [ ]:
from jsonschema.exceptions import ValidationError
from jsonschema.validators import validate

stac_catalog_files = []
stac_collection_files = []
stac_item_files = []
for root, _, files in os.walk(CATALOG_ROOT_DIR):
    for file in files:
        file_path = os.path.join(root, file)
        if file == "catalog.json":
            stac_catalog_files.append(file_path)
        elif file == "collection.json":
            stac_collection_files.append(file_path)
        elif file.startswith("item-") and file.endswith(".json"):
            stac_item_files.append(file_path)

assert stac_catalog_files
assert stac_collection_files
assert stac_item_files

stac_file = None
schema_file = None
try:
    for files, schemas in [
        (stac_catalog_files, STAC_CATALOG_SCHEMAS),
        (stac_collection_files, STAC_COLLECTION_SCHEMAS),
        (stac_item_files, STAC_ITEM_SCHEMAS),
    ]:
        for schema_file in schemas:
            schema = load_json(schema_file)
            for stac_file in files:
                content = load_json(stac_file)
                validate(content, schema)
except ValidationError as exc:
    raise AssertionError(f"Failed [{stac_file}] validation against [{schema_file}]") from exc
